<h2 style="color:#0078D4;">Notebook Information</h2>

| | |
|:---|:---|
| **Notebook 1** | Monitor Hub Analysis |
| **Author** | Sanmi Ibitoye |
| **Email** | Sanmi.Ibitoye@leit.ltd |
| **Summary** | Runs the MonitorHubPipeline and analyzes exported CSV/Parquet outputs. Works in both Microsoft Fabric notebooks (paths auto-resolve under `/lakehouse/default/Files/`) and local development (writes under `exports/`). |
| **Date** | Last Updated: 18 - 12 - 2025 |

---

### Key Behaviors
- **CSV-based reports**: The pipeline writes timestamped reports like `activities_master_YYYYMMDD_HHMMSS.csv`
- **Strict auth when SP is provided**: If you set `AZURE_CLIENT_ID`/`AZURE_CLIENT_SECRET`/`AZURE_TENANT_ID` and they're wrong, auth fails rather than silently switching identities
- **Tenant-wide + fallback**: When `TENANT_WIDE=True`, Power BI Admin APIs are attempted first and automatically fall back to member-only scope on 401/403
- **Caching**: Daily extraction skips API calls if daily files already exist, and detailed job history uses an ~8 hour cache

### How to Use
1. Configure credentials (Service Principal or Fabric identity)
2. Set `DAYS_TO_ANALYZE`, `TENANT_WIDE`, and `OUTPUT_DIR` in the config cell
3. Run the pipeline cell; then run the analysis cells

In [ ]:
# SETUP LOCAL PATH (For Local Development)
import sys
import os
from pathlib import Path

# Add the src directory to sys.path to allow importing the local package
# This is necessary when running locally without installing the package
current_dir = Path(os.getcwd())

# Check if we are in notebooks directory
if current_dir.name == "notebooks":
    src_path = current_dir.parent / "src"
else:
    # Assume we are in project root
    src_path = current_dir / "src"

if src_path.exists() and str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"Added {src_path} to sys.path")

In [ ]:
# Package / environment verification (safe: no Azure/API imports)
from importlib.metadata import PackageNotFoundError, version
import importlib
import usf_fabric_monitoring
from usf_fabric_monitoring.core.utils import resolve_path

try:
    pkg_version = getattr(usf_fabric_monitoring, "__version__", None) or version("usf_fabric_monitoring")
except PackageNotFoundError:
    pkg_version = "unknown"

print(f"usf_fabric_monitoring version: {pkg_version}")
print(f"Resolved output dir example: {resolve_path('exports/monitor_hub_analysis')}")

# Optional dependency presence (does not import pipeline/auth)
for mod in ["azure.identity", "azure.core", "pyspark", "delta", "notebookutils"]:
    try:
        importlib.import_module(mod)
        print(f"OK: import {mod}")
    except Exception as e:
        print(f"SKIP: import {mod} ({type(e).__name__}: {e})")

In [ ]:
import os

In [ ]:
# Package / environment verification (safe: no Azure/API imports)
from importlib.metadata import PackageNotFoundError, version
import importlib
import usf_fabric_monitoring
from usf_fabric_monitoring.core.utils import resolve_path

try:
    pkg_version = getattr(usf_fabric_monitoring, "__version__", None) or version("usf_fabric_monitoring")
except PackageNotFoundError:
    pkg_version = "unknown"

print(f"usf_fabric_monitoring version: {pkg_version}")
print(f"Resolved output dir example: {resolve_path('exports/monitor_hub_analysis')}")

# Optional dependency presence (does not import pipeline/auth)
for mod in ["azure.identity", "azure.core", "pyspark", "delta", "notebookutils"]:
    try:
        importlib.import_module(mod)
        print(f"OK: import {mod}")
    except Exception as e:
        print(f"SKIP: import {mod} ({type(e).__name__}: {e})")

In [ ]:
import os
import base64
import json
from dotenv import load_dotenv

# --- CREDENTIAL MANAGEMENT ---

# Option 1: Load from .env file (Lakehouse or Local)
# We check the Lakehouse path first, then fallback to local .env
LAKEHOUSE_ENV_PATH = "/lakehouse/default/Files/dot_env_files/.env"
LOCAL_ENV_PATH = ".env"

# Force override=True to ensure we pick up changes to the file even if env vars are already set
if os.path.exists(LAKEHOUSE_ENV_PATH):
    print(f"Loading configuration from Lakehouse: {LAKEHOUSE_ENV_PATH}")
    load_dotenv(LAKEHOUSE_ENV_PATH, override=True)
elif os.path.exists(LOCAL_ENV_PATH):
    print(f"Loading configuration from Local: {os.path.abspath(LOCAL_ENV_PATH)}")
    load_dotenv(LOCAL_ENV_PATH, override=True)
else:
    print(f"Warning: No .env file found at {LAKEHOUSE_ENV_PATH} or {LOCAL_ENV_PATH}")

# Verify credentials are present
required_vars = ["AZURE_CLIENT_ID", "AZURE_CLIENT_SECRET", "AZURE_TENANT_ID"]
missing = [v for v in required_vars if not os.getenv(v)]

print("\n IDENTITY CHECK:")
if missing:
    print(f" Missing required environment variables: {', '.join(missing)}")
    print("     System will fallback to DefaultAzureCredential (User Identity or Managed Identity)")
else:
    client_id = os.getenv("AZURE_CLIENT_ID")
    masked_id = f"{client_id[:4]}...{client_id[-4:]}" if client_id and len(client_id) > 8 else "********"
    print(f" Service Principal Configured in Environment")
    print(f"   Client ID: {masked_id}")
    print(f"   Tenant ID: {os.getenv('AZURE_TENANT_ID')}")

# --- TOKEN IDENTITY INSPECTION ---
# This block decodes the actual token being used to prove identity
try:
    from usf_fabric_monitoring.core.auth import create_authenticator_from_env
    auth = create_authenticator_from_env()
    token = auth.get_fabric_token()
    
    # Decode JWT (no signature verification needed for inspection)
    parts = token.split('.')
    if len(parts) > 1:
        # Add padding if needed
        payload_part = parts[1]
        padded = payload_part + '=' * (4 - len(payload_part) % 4)
        decoded = base64.urlsafe_b64decode(padded)
        claims = json.loads(decoded)
        
        print("\n  ACTIVE TOKEN IDENTITY:")
        if 'upn' in claims:
            print(f"   User Principal Name: {claims['upn']}")
            print("    You are logged in as a USER.")
        elif 'appid' in claims:
            print(f"   Application ID: {claims['appid']}")
            if client_id and claims['appid'] == client_id:
                print("    You are logged in as the CONFIGURED SERVICE PRINCIPAL.")
            else:
                print("    You are logged in as a DIFFERENT Service Principal/Managed Identity.")
        else:
            print(f"   Subject: {claims.get('sub', 'Unknown')}")
            
        print(f"   Audience: {claims.get('aud', 'Unknown')}")
except Exception as e:
    print(f"\n  Could not inspect token identity: {e}")

In [ ]:
# Run configuration
import os
from usf_fabric_monitoring.core.utils import resolve_path

# Days of history to analyze (API max defaults to 28; see MAX_HISTORICAL_DAYS)
DAYS_TO_ANALYZE = int(os.getenv("DEFAULT_ANALYSIS_DAYS", "7"))

# Scope: tenant-wide uses Power BI Admin APIs and auto-falls back to member-only on 401/403
TENANT_WIDE = os.getenv("TENANT_WIDE", "1") == "1"

# Output directory for reports and parquet (relative paths auto-resolve in Fabric)
OUTPUT_DIR = os.getenv("EXPORT_DIRECTORY", "exports/monitor_hub_analysis")

# Optional network knobs (helps avoid long retry backoffs in interactive runs)
os.environ.setdefault("API_REQUEST_TIMEOUT", "30")
os.environ.setdefault("MAX_RETRIES", "2")
os.environ.setdefault("RETRY_BACKOFF_FACTOR", "1")

# Notebook-level cache control
FORCE_REFRESH = os.getenv("FORCE_REFRESH", "0") == "1"

resolved_output_dir = resolve_path(OUTPUT_DIR)

print(f"DAYS_TO_ANALYZE: {DAYS_TO_ANALYZE} (max {os.getenv('MAX_HISTORICAL_DAYS', '28')})")
print(f"TENANT_WIDE: {TENANT_WIDE}")
print(f"FORCE_REFRESH: {FORCE_REFRESH}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")
print(f"Resolved output dir: {resolved_output_dir}")

In [ ]:
# Smart Data Extraction with 8-Hour Cache Logic
import os
import glob
from datetime import datetime, timedelta
from pathlib import Path
from usf_fabric_monitoring.core.utils import resolve_path

def check_recent_extraction(output_dir: str, hours_threshold: int = 8):
    """Check if reports were generated within the threshold hours."""
    try:
        resolved_dir = resolve_path(output_dir)

        csv_pattern = os.path.join(str(resolved_dir), "activities_master_*.csv")
        csv_files = glob.glob(csv_pattern)

        # Backward-compatible: if the default output dir changed but legacy cache exists, use it.
        if (
            not csv_files
            and output_dir == "exports/monitor_hub_analysis"
            and glob.glob(str(resolve_path("monitor_hub_analysis") / "activities_master_*.csv"))
        ):
            print("Found recent extraction under legacy OUTPUT_DIR; reusing it to avoid re-running APIs.")
            output_dir = "monitor_hub_analysis"
            resolved_dir = resolve_path(output_dir)
            csv_pattern = os.path.join(str(resolved_dir), "activities_master_*.csv")
            csv_files = glob.glob(csv_pattern)

        if not csv_files:
            print("No previous extraction found")
            return False, None, output_dir

        latest_file = None
        latest_time = None
        for csv_file in csv_files:
            file_time = os.path.getctime(csv_file)
            if latest_time is None or file_time > latest_time:
                latest_time = file_time
                latest_file = csv_file

        file_time = datetime.fromtimestamp(latest_time)
        time_diff = datetime.now() - file_time

        print(f"Latest extraction: {os.path.basename(latest_file)}")
        print(f"Extraction time: {file_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Time since extraction: {time_diff}")

        if time_diff < timedelta(hours=hours_threshold):
            print(f"Using cached data (within {hours_threshold} hours)")
            return True, latest_file, output_dir
        else:
            print(f"Cache expired (older than {hours_threshold} hours)")
            return False, latest_file, output_dir

    except Exception as e:
        print(f"Error checking cache: {e}")
        return False, None, output_dir

print("CHECKING FOR RECENT DATA EXTRACTION...")
use_cache, cache_file, effective_output_dir = check_recent_extraction(OUTPUT_DIR, hours_threshold=8)
if effective_output_dir != OUTPUT_DIR:
    OUTPUT_DIR = effective_output_dir
    resolved_output_dir = resolve_path(OUTPUT_DIR)
    print(f"Using OUTPUT_DIR: {OUTPUT_DIR} -> {resolved_output_dir}")

if use_cache and not FORCE_REFRESH:
    print("USING CACHED DATA - SKIPPING EXTRACTION")

    # Load cached pipeline summary for results display
    try:
        resolved_dir = resolve_path(OUTPUT_DIR)
        summary_pattern = os.path.join(str(resolved_dir), "pipeline_summary_*.json")
        summary_files = glob.glob(summary_pattern)

        if summary_files:
            import json
            latest_summary = None
            latest_time = None
            for summary_file in summary_files:
                file_time = os.path.getctime(summary_file)
                if latest_time is None or file_time > latest_time:
                    latest_time = file_time
                    latest_summary = summary_file

            with open(latest_summary, "r", encoding="utf-8") as f:
                cached_results = json.load(f)

            results = {
                "status": "success",
                "summary": cached_results,
                "report_files": {},
                "cached": True,
            }

            print("Cached Analysis Summary:")

            def safe_format(key, value):
                try:
                    if key == "success_rate" and isinstance(value, (int, float)):
                        return f"  {key.replace('_', ' ').title()}: {value:.1f}%"
                    elif key in ["total_activities", "analysis_period_days"] and value is not None:
                        return f"  {key.replace('_', ' ').title()}: {value:,}"
                    elif value is not None:
                        return f"  {key.replace('_', ' ').title()}: {value}"
                    else:
                        return f"  {key.replace('_', ' ').title()}: N/A"
                except (ValueError, TypeError):
                    return f"  {key.replace('_', ' ').title()}: {value}"

            for k in ["total_activities", "analysis_period_days", "success_rate", "total_workspaces", "total_items"]:
                if k in cached_results:
                    print(safe_format(k, cached_results.get(k)))
        else:
            results = {"status": "success", "cached": True, "summary": {"note": "Using cached data"}}
            print("Cached Analysis Summary: Using recent extraction (summary file not found)")
    except Exception as e:
        print(f"Could not load cached summary: {e}")
        results = {"status": "success", "cached": True}
else:
    if FORCE_REFRESH:
        print("FORCE_REFRESH is enabled: running fresh extraction")
    else:
        print("No fresh cache found: running extraction")

    from usf_fabric_monitoring.core.pipeline import MonitorHubPipeline
    pipeline = MonitorHubPipeline(OUTPUT_DIR)
    results = pipeline.run_complete_analysis(days=DAYS_TO_ANALYZE, tenant_wide=TENANT_WIDE)

print("\nPIPELINE COMPLETE")

## 5. Advanced Analysis & Visualization (Spark)

The following cells use PySpark to load the exported/enriched monitoring data and provide visualizations of failures, error codes, and trends.

In [ ]:
# 1. Setup Spark & Paths for Smart Merge Enhanced Data
import os
import glob
from usf_fabric_monitoring.core.utils import resolve_path

print(" INITIALIZING SPARK FOR SMART MERGE ENHANCED DATA ANALYSIS")

# Initialize Spark Session (if not already active)
spark = None
try:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import (
        col, to_timestamp, when, count, desc, lit, unix_timestamp, coalesce, 
        abs as abs_val, split, initcap, regexp_replace, element_at, substring, 
        avg, max, min, to_date, countDistinct, collect_list
    )
    from pyspark.sql.types import StructType, StructField, StringType, DoubleType

    if 'spark' not in locals() or spark is None:
        print(" Initializing Spark Session for Smart Merge data...")
        spark = SparkSession.builder \
            .appName("FabricSmartMergeAnalysis") \
            .getOrCreate()
        print(f" Spark Session Created: {spark.version}")
        print("    Ready for Smart Merge enhanced data analysis")
except ImportError:
    print(" PySpark not installed or configured. Skipping Spark-based analysis.")
except Exception as e:
    print(f" Failed to initialize Spark: {e}. Skipping Spark-based analysis.")

# Resolve the output directory to an absolute path
# This ensures that if you used a relative path like "monitor_hub_analysis",
# it is correctly resolved to "/lakehouse/default/Files/monitor_hub_analysis" for Spark.
resolved_output_dir = str(resolve_path(OUTPUT_DIR))

BASE_PATH = os.path.join(resolved_output_dir, "fabric_item_details")
AUDIT_LOG_PATH = os.path.join(resolved_output_dir, "raw_data/daily")

print(f"\n Smart Merge Enhanced Data Paths:")
print(f"  - Item Details: {BASE_PATH}")
print(f"  - Audit Logs:   {AUDIT_LOG_PATH}")
print("    All paths contain Smart Merge enhanced data with 100% duration recovery")

In [ ]:
# 2. Load Smart Merge Enhanced Data from CSV (Aggregated Reports) - 22-COLUMN SCHEMA VALIDATION

import os
import glob
from pyspark.sql.functions import col, to_timestamp, unix_timestamp, coalesce, initcap, regexp_replace, element_at, split, when, lit, to_date
from pyspark.sql.types import StringType
from usf_fabric_monitoring.core.utils import is_fabric_environment

def convert_to_spark_path(local_path: str) -> str:
    """Convert local filesystem path to Spark-compatible path.
    
    In Microsoft Fabric, local paths like /lakehouse/default/Files/...
    need to be converted to abfss:// URIs for Spark to read them.
    
    Args:
        local_path: Local filesystem path (e.g., /lakehouse/default/Files/exports/...)
        
    Returns:
        Spark-compatible path (abfss:// in Fabric, unchanged locally)
    """
    if not is_fabric_environment():
        return local_path  # Local development - use path as-is
    
    # In Fabric, convert /lakehouse/default/Files/... to Files/...
    # Spark automatically resolves relative paths from the Lakehouse root
    lakehouse_prefix = "/lakehouse/default/Files/"
    if local_path.startswith(lakehouse_prefix):
        # Return the relative path from Files/ - Spark handles this correctly
        relative_path = local_path[len("/lakehouse/default/"):]
        return relative_path
    
    # If path doesn't match expected pattern, return as-is
    return local_path

def load_smart_merge_csv_data():
    """Loads the Smart Merge enhanced activity data from CSV reports with schema validation.
    
    Only loads CSV files that match the expected 22-column schema:
    activity_id, workspace_id, workspace_name, item_id, item_name, item_type, activity_type,
    status, start_time, end_time, date, hour, duration_seconds, duration_minutes, submitted_by,
    created_by, last_updated_by, domain, location, object_url, failure_reason, error_message
    
    Enrichment provides:
    - 100% duration data recovery through advanced correlation
    - Enhanced accuracy in performance metrics
    - Intelligent gap filling for missing information
    - Comprehensive activity lifecycle tracking
    """
    try:
        # Expected 22-column schema
        expected_columns = [
            'activity_id', 'workspace_id', 'workspace_name', 'item_id', 'item_name', 'item_type',
            'activity_type', 'status', 'start_time', 'end_time', 'date', 'hour',
            'duration_seconds', 'duration_minutes', 'submitted_by', 'created_by',
            'last_updated_by', 'domain', 'location', 'object_url', 'failure_reason', 'error_message'
        ]
        
        # Find all CSV files matching the pattern using local filesystem
        csv_pattern = os.path.join(resolved_output_dir, "activities_master_*.csv")
        all_csv_files = glob.glob(csv_pattern)
        
        if not all_csv_files:
            print(f" No CSV files found matching pattern: {csv_pattern}")
            return None
        
        print(f" Found {len(all_csv_files)} CSV file(s) - validating schemas...")
        print(f"   Expected schema: 22 columns")
        
        # Validate each file's schema using local filesystem read
        valid_files = []
        invalid_files = []
        
        for csv_file in all_csv_files:
            file_name = os.path.basename(csv_file)
            
            # Read just the header to check schema (uses local filesystem)
            with open(csv_file, 'r') as f:
                header = f.readline().strip()
                columns = [c.strip() for c in header.split(',')]
            
            if len(columns) == 22 and set(columns) == set(expected_columns):
                valid_files.append(csv_file)
                print(f"    {file_name}: Valid (22 columns)")
            else:
                invalid_files.append((csv_file, len(columns)))
                print(f"     {file_name}: INVALID ({len(columns)} columns - SKIPPING)")
        
        if invalid_files:
            print(f"\n  WARNING: {len(invalid_files)} file(s) do not match the expected 22-column schema:")
            for invalid_file, col_count in invalid_files:
                file_name = os.path.basename(invalid_file)
                print(f"    {file_name} has {col_count} columns (expected 22)")
            print(f"    These files will be EXCLUDED from the analysis.")
            print(f"    Consider deleting old files or re-running the pipeline to regenerate them.")
        
        if not valid_files:
            print(f"\n ERROR: No valid CSV files found with the expected 22-column schema!")
            print(f"   Please re-run the pipeline to generate updated CSV files.")
            return None
        
        print(f"\n Loading {len(valid_files)} valid CSV file(s) with 22-column schema...")
        
        # Convert local paths to Spark-compatible paths
        # In Fabric, Spark needs relative paths from /lakehouse/default/ not absolute paths
        spark_files = [convert_to_spark_path(f) for f in valid_files]
        
        if is_fabric_environment():
            print(f"   Fabric environment detected - using relative paths for Spark")
            for local_path, spark_path in zip(valid_files, spark_files):
                print(f"     {os.path.basename(local_path)} -> {spark_path}")
        
        # Load all valid CSV files using Spark-compatible paths
        df = spark.read.option("header", "true").option("inferSchema", "false").csv(spark_files)
        
        # Enhanced data validation for Smart Merge features
        total_records = df.count()
        print(f"    Total records loaded: {total_records:,}")
        
        if total_records == 0:
            print(" No data found in valid CSV files")
            return None
        
        # Verify the loaded DataFrame has correct schema
        actual_columns = df.columns
        print(f"    DataFrame columns ({len(actual_columns)}): {', '.join(actual_columns)}")
        
        if len(actual_columns) != 22:
            print(f"    WARNING: DataFrame has {len(actual_columns)} columns, expected 22")
        
        # Check for enhanced Smart Merge columns
        smart_merge_cols = ['workspace_name', 'failure_reason', 'error_message']
        missing_cols = [c for c in smart_merge_cols if c not in actual_columns]
        
        if missing_cols:
            print(f"    WARNING: Missing Smart Merge columns: {', '.join(missing_cols)}")
        else:
            print(f"    All Smart Merge enhanced columns present")
        
        # Check for enhanced duration data
        duration_cols = [c for c in actual_columns if 'duration' in c.lower()]
        if duration_cols:
            print(f"    Duration columns detected: {', '.join(duration_cols)}")
            print("    Smart Merge duration enhancement active")
        
        print(f"    Successfully loaded aggregated data from {len(valid_files)} file(s)")
        return df
            
    except Exception as e:
        print(f" Could not load Smart Merge enhanced CSV data: {str(e)}")
        print("   Tip: Ensure the pipeline ran successfully and generated enhanced CSV reports.")
        import traceback
        traceback.print_exc()
        return None

# Execute Smart Merge Enhanced Loading
print(" LOADING SMART MERGE ENHANCED DATA (LATEST FILE)...")
complete_df = load_smart_merge_csv_data()

if complete_df:
    record_count = complete_df.count()
    print(f"\n Successfully loaded {record_count:,} Smart Merge enhanced records.")
    print("    Data includes 100% duration recovery and advanced correlation")
    
    # Verify we have workspace_name column from CSV
    if 'workspace_name' in complete_df.columns:
        print(f"    workspace_name column found in CSV data")
    else:
        print(f"    WARNING: workspace_name column NOT found!")
        print(f"    You may need to re-run the pipeline with the latest version")
    
    # Show status breakdown
    status_breakdown = complete_df.groupBy("status").count().collect()
    print("\n    Status Breakdown:")
    for row in status_breakdown:
        print(f"      {row['status']}: {row['count']:,}")
    
else:
    print(" No Smart Merge enhanced data found.")
    print(f"    Checked path: {resolved_output_dir}")
    # Let's also check what files actually exist
    try:
        import glob
        all_csv_files = glob.glob(os.path.join(resolved_output_dir, "*.csv"))
        print(f"    Available CSV files: {[os.path.basename(f) for f in all_csv_files]}")
    except Exception as list_error:
        print(f"    Could not list files: {list_error}")

In [ ]:
# 3. Data Validation & Column Check
print("=" * 80)
print(" DATA VALIDATION - VERIFYING CSV COLUMNS")
print("=" * 80)

if 'complete_df' in dir() and complete_df is not None:
    total_records = complete_df.count()
    print(f"\n Total Records Loaded: {total_records:,}")
    
    print(f"\n Available Columns ({len(complete_df.columns)}):")
    for idx, col_name in enumerate(complete_df.columns, 1):
        print(f"   {idx:2d}. {col_name}")
    
    # Verify critical columns exist
    critical_columns = ['workspace_id', 'workspace_name', 'item_name', 'item_type', 
                       'activity_type', 'status', 'start_time', 'end_time', 
                       'duration_seconds', 'failure_reason', 'error_message']
    
    print(f"\n Critical Column Check:")
    missing_columns = []
    for col_name in critical_columns:
        if col_name in complete_df.columns:
            print(f"    {col_name}")
        else:
            print(f"    {col_name} - MISSING!")
            missing_columns.append(col_name)
    
    if missing_columns:
        print(f"\n  WARNING: {len(missing_columns)} critical columns are missing!")
        print(f"   Missing: {', '.join(missing_columns)}")
    else:
        print(f"\n All critical columns present!")
    
    # Show status breakdown
    print(f"\n Status Breakdown:")
    status_summary = complete_df.groupBy("status").count().orderBy(col("count").desc()).collect()
    for row in status_summary:
        status_name = row['status'] if row['status'] else 'Unknown'
        count = row['count']
        percentage = (count / total_records * 100) if total_records > 0 else 0
        print(f"   {status_name:15s}: {count:10,} ({percentage:5.2f}%)")
    
    # Check workspace_name data quality
    if 'workspace_name' in complete_df.columns:
        null_workspace_names = complete_df.filter(col("workspace_name").isNull() | (col("workspace_name") == "")).count()
        valid_workspace_names = total_records - null_workspace_names
        print(f"\n Workspace Name Data Quality:")
        print(f"   Valid workspace names: {valid_workspace_names:,} ({valid_workspace_names/total_records*100:.1f}%)")
        print(f"   Null/Empty: {null_workspace_names:,} ({null_workspace_names/total_records*100:.1f}%)")
        
        # Show sample workspace names
        print(f"\n Sample Workspace Names:")
        sample_workspaces = complete_df.select("workspace_name", "status").limit(10).collect()
        for row in sample_workspaces:
            ws_name = row['workspace_name'] if row['workspace_name'] else "NULL"
            status = row['status'] if row['status'] else "Unknown"
            print(f"   {ws_name:50s} [{status}]")
    
    print(f"\n" + "=" * 80)
    print(" DATA VALIDATION COMPLETE")
    print("=" * 80)
    
else:
    print(" complete_df not loaded!")
    print("    Run Cell 11 first to load the CSV data")


In [ ]:
# 4. Overall Statistics Summary
print("=" * 80)
print(" OVERALL ACTIVITY STATISTICS")
print("=" * 80)

if complete_df:
    # Import required functions explicitly
    from pyspark.sql.functions import col, count, countDistinct, avg, sum as spark_sum, desc, when, max as spark_max, min as spark_min
    
    total_records = complete_df.count()
    print(f"\n Dataset Overview:")
    print(f"   Total Activities: {total_records:,}")
    
    # Status breakdown with percentages
    print(f"\n Status Distribution:")
    status_df = complete_df.groupBy("status").agg(count("*").alias("count"))
    status_results = status_df.collect()
    
    for row in status_results:
        status_name = row['status'] if row['status'] else 'Unknown'
        count_val = row['count']
        pct = (count_val / total_records * 100) if total_records > 0 else 0
        print(f"   {status_name:15s}: {count_val:10,} ({pct:5.2f}%)")
    
    # Workspace statistics
    print(f"\n Workspace Statistics:")
    unique_workspaces = complete_df.select("workspace_name").filter(col("workspace_name").isNotNull()).distinct().count()
    print(f"   Unique Workspaces: {unique_workspaces:,}")
    
    # Item statistics
    print(f"\n Item Statistics:")
    unique_items = complete_df.select("item_name").filter(col("item_name").isNotNull()).distinct().count()
    unique_item_types = complete_df.select("item_type").filter(col("item_type").isNotNull()).distinct().count()
    print(f"   Unique Items: {unique_items:,}")
    print(f"   Unique Item Types: {unique_item_types:,}")
    
    # Activity type statistics
    print(f"\n  Activity Type Statistics:")
    unique_activity_types = complete_df.select("activity_type").filter(col("activity_type").isNotNull()).distinct().count()
    print(f"   Unique Activity Types: {unique_activity_types:,}")
    
    # User statistics
    print(f"\n User Statistics:")
    unique_users = complete_df.select("submitted_by").filter(
        (col("submitted_by").isNotNull()) & 
        (col("submitted_by") != "System") & 
        (col("submitted_by") != "")
    ).distinct().count()
    print(f"   Unique Active Users: {unique_users:,}")
    
    # Duration statistics
    print(f"\n⏱  Duration Statistics:")
    duration_df = complete_df.filter(col("duration_seconds").isNotNull() & (col("duration_seconds").cast("double") > 0))
    duration_count = duration_df.count()
    
    if duration_count > 0:
        duration_stats = duration_df.agg(
            avg(col("duration_seconds").cast("double")).alias("avg_duration"),
            spark_max(col("duration_seconds").cast("double")).alias("max_duration"),
            spark_min(col("duration_seconds").cast("double")).alias("min_duration")
        ).collect()[0]
        
        print(f"   Activities with Duration: {duration_count:,} ({duration_count/total_records*100:.1f}%)")
        print(f"   Average Duration: {duration_stats['avg_duration']:.1f}s")
        print(f"   Max Duration: {duration_stats['max_duration']:.1f}s")
        print(f"   Min Duration: {duration_stats['min_duration']:.1f}s")
    else:
        print(f"   No duration data available")
    
    print(f"\n" + "=" * 80)

else:
    print(" complete_df not available")

In [ ]:
# 5. Workspace Activity Analysis (Using workspace_name from CSV)
print("=" * 80)
print(" WORKSPACE ACTIVITY ANALYSIS")
print("=" * 80)

if complete_df:
    from pyspark.sql.functions import col, count, countDistinct, desc
    
    # Top workspaces by total activity
    print(f"\n TOP 20 MOST ACTIVE WORKSPACES:")
    print("-" * 80)
    
    workspace_activity = (complete_df
                         .filter(col("workspace_name").isNotNull())
                         .groupBy("workspace_name")
                         .agg(
                             count("*").alias("total_activities"),
                             countDistinct("item_name").alias("unique_items"),
                             countDistinct("activity_type").alias("activity_types"),
                             countDistinct("submitted_by").alias("unique_users")
                         )
                         .orderBy(desc("total_activities"))
                         .limit(20))
    
    top_workspaces = workspace_activity.collect()
    for idx, row in enumerate(top_workspaces, 1):
        ws_name = row['workspace_name']
        activities = row['total_activities']
        items = row['unique_items']
        types = row['activity_types']
        users = row['unique_users']
        print(f"  {idx:2d}. {ws_name:45s}  {activities:8,} activities  {items:4,} items  {types:3,} types  {users:4,} users")
    
    print(f"\n" + "=" * 80)

else:
    print(" complete_df not available")

In [ ]:
# 6. Failure Analysis by Workspace (FIXED - Using workspace_name from CSV)
print("=" * 80)
print(" WORKSPACE FAILURE ANALYSIS")
print("=" * 80)

if complete_df:
    from pyspark.sql.functions import col, count, countDistinct, desc
    
    # Filter for failures
    failures_df = complete_df.filter(col("status") == "Failed")
    failure_count = failures_df.count()
    
    print(f"\n Total Failures: {failure_count:,}")
    
    if failure_count > 0:
        # Failures by workspace
        print(f"\n TOP 20 WORKSPACES WITH FAILURES:")
        print("-" * 80)
        
        workspace_failures = (failures_df
                             .filter(col("workspace_name").isNotNull())
                             .groupBy("workspace_name")
                             .agg(
                                 count("*").alias("failure_count"),
                                 countDistinct("item_name").alias("failed_items"),
                                 countDistinct("activity_type").alias("failure_types")
                             )
                             .orderBy(desc("failure_count"))
                             .limit(20))
        
        top_failure_workspaces = workspace_failures.collect()
        
        if len(top_failure_workspaces) > 0:
            for idx, row in enumerate(top_failure_workspaces, 1):
                ws_name = row['workspace_name']
                failures = row['failure_count']
                items = row['failed_items']
                types = row['failure_types']
                print(f"  {idx:2d}. {ws_name:45s}  {failures:6,} failures  {items:4,} items  {types:3,} types")
        else:
            print("   No failures with workspace names found")
        
        # Check for failures without workspace names
        failures_no_workspace = failures_df.filter(col("workspace_name").isNull() | (col("workspace_name") == "")).count()
        
        if failures_no_workspace > 0:
            print(f"\n  Failures without workspace name: {failures_no_workspace:,} ({failures_no_workspace/failure_count*100:.1f}%)")
            print(f"   These may be system-level or infrastructure failures")
        
        # Failure types distribution
        print(f"\n  FAILURE TYPES DISTRIBUTION:")
        print("-" * 80)
        
        failure_types = (failures_df
                        .groupBy("activity_type")
                        .agg(count("*").alias("failure_count"))
                        .orderBy(desc("failure_count"))
                        .limit(10))
        
        failure_type_results = failure_types.collect()
        for idx, row in enumerate(failure_type_results, 1):
            activity_type = row['activity_type'] if row['activity_type'] else "Unknown"
            failures = row['failure_count']
            pct = (failures / failure_count * 100) if failure_count > 0 else 0
            print(f"  {idx:2d}. {activity_type:35s}  {failures:6,} failures ({pct:5.1f}%)")
        
        # Top failing items
        print(f"\n TOP 15 FAILING ITEMS:")
        print("-" * 80)
        
        failing_items = (failures_df
                        .filter(col("item_name").isNotNull())
                        .groupBy("workspace_name", "item_name", "item_type")
                        .agg(count("*").alias("failure_count"))
                        .orderBy(desc("failure_count"))
                        .limit(15))
        
        failing_item_results = failing_items.collect()
        for idx, row in enumerate(failing_item_results, 1):
            ws_name = row['workspace_name'] if row['workspace_name'] else "Unknown Workspace"
            item_name = row['item_name']
            item_type = row['item_type'] if row['item_type'] else "Unknown"
            failures = row['failure_count']
            print(f"  {idx:2d}. {item_name:30s} ({item_type:15s})  {ws_name:25s}  {failures:5,} failures")
        
    else:
        print(" No failures found in the dataset")
    
    print(f"\n" + "=" * 80)

else:
    print(" complete_df not available")

In [ ]:
# 7. User Activity & Failure Analysis
print("=" * 80)
print(" USER ACTIVITY ANALYSIS")
print("=" * 80)

if complete_df:
    from pyspark.sql.functions import col, count, countDistinct, desc, when, sum as spark_sum
    
    # Based on diagnostic: submitted_by has 95.74% data, created_by is 100% NULL
    user_column = 'submitted_by'
    print(f"\n Using '{user_column}' column for user analysis")
    print("   (created_by and last_updated_by are 100% NULL in this dataset)")
    
    # Filter out system users and nulls
    user_activities = complete_df.filter(
        (col(user_column).isNotNull()) & 
        (col(user_column) != "System") & 
        (col(user_column) != "")
    )
    
    total_user_activities = user_activities.count()
    unique_users = user_activities.select(user_column).distinct().count()
    
    print(f"\n User Activity Overview:")
    print(f"   Total User Activities: {total_user_activities:,}")
    print(f"   Unique Active Users: {unique_users:,}")
    
    if total_user_activities > 0:
        # Top active users - HANDLES DUPLICATES with groupBy aggregation
        print(f"\n TOP 20 MOST ACTIVE USERS:")
        print("-" * 80)
        
        # groupBy automatically handles duplicates by aggregating them
        # countDistinct ensures we count unique workspaces/items per user
        top_users = (user_activities
                    .groupBy(user_column)
                    .agg(
                        count("*").alias("total_activities"),  # Total activities (including duplicates if any)
                        countDistinct("workspace_name").alias("workspaces"),  # Unique workspaces
                        countDistinct("item_name").alias("unique_items"),  # Unique items
                        spark_sum(when(col("status") == "Failed", 1).otherwise(0)).alias("failures"),
                        spark_sum(when(col("status") == "Succeeded", 1).otherwise(0)).alias("successes")
                    )
                    .orderBy(desc("total_activities"))
                    .limit(20))
        
        top_user_results = top_users.collect()
        
        if len(top_user_results) > 0:
            for idx, row in enumerate(top_user_results, 1):
                user = row[user_column]
                activities = row['total_activities']
                workspaces = row['workspaces']
                items = row['unique_items']
                failures = row['failures']
                successes = row['successes']
                success_rate = (successes / activities * 100) if activities > 0 else 0
                print(f"  {idx:2d}. {user:40s}  {activities:7,} activities  {workspaces:3,} WS  {items:4,} items  {failures:5,} fails  {success_rate:5.1f}% success")
        else:
            print("   No user data available")
    else:
        print("\n    No user activities found (all records may be System or null)")
    
    # Users with most failures
    if total_user_activities > 0:
        user_failures = user_activities.filter(col("status") == "Failed")
        user_failure_count = user_failures.count()
        
        if user_failure_count > 0:
            print(f"\n TOP 10 USERS WITH MOST FAILURES:")
            print("-" * 80)
            
            # groupBy with countDistinct handles duplicates properly
            users_with_failures = (user_failures
                                  .groupBy(user_column)
                                  .agg(
                                      count("*").alias("failure_count"),  # Total failures per user
                                      countDistinct("workspace_name").alias("affected_workspaces"),  # Unique workspaces
                                      countDistinct("item_name").alias("failed_items")  # Unique items
                                  )
                                  .orderBy(desc("failure_count"))
                                  .limit(10))
            
            user_failure_results = users_with_failures.collect()
            
            if len(user_failure_results) > 0:
                for idx, row in enumerate(user_failure_results, 1):
                    user = row[user_column]
                    failures = row['failure_count']
                    workspaces = row['affected_workspaces']
                    items = row['failed_items']
                    print(f"  {idx:2d}. {user:40s}  {failures:6,} failures  {workspaces:3,} WS  {items:4,} items")
            else:
                print("   No failure data available")
        else:
            print(f"\n No failures found for users")
    
    print(f"\n" + "=" * 80)

else:
    print(" complete_df not available")

In [ ]:
# 8. Error & Failure Reason Analysis
print("=" * 80)
print(" ERROR & FAILURE REASON ANALYSIS")
print("=" * 80)

if complete_df:
    from pyspark.sql.functions import col, count, desc
    
    failures_df = complete_df.filter(col("status") == "Failed")
    failure_count = failures_df.count()
    
    if failure_count > 0:
        # Check if error columns exist and have data
        has_failure_reason = 'failure_reason' in complete_df.columns
        has_error_message = 'error_message' in complete_df.columns
        
        if has_failure_reason:
            # Failure reason distribution
            print(f"\n FAILURE REASONS:")
            print("-" * 80)
            
            failure_reasons = (failures_df
                             .filter(col("failure_reason").isNotNull() & (col("failure_reason") != ""))
                             .groupBy("failure_reason")
                             .agg(count("*").alias("count"))
                             .orderBy(desc("count"))
                             .limit(15))
            
            reason_results = failure_reasons.collect()
            
            if len(reason_results) > 0:
                for idx, row in enumerate(reason_results, 1):
                    reason = row['failure_reason']
                    count_val = row['count']
                    pct = (count_val / failure_count * 100) if failure_count > 0 else 0
                    # Truncate long reasons
                    reason_display = reason[:70] + "..." if len(reason) > 70 else reason
                    print(f"  {idx:2d}. {reason_display:73s}  {count_val:5,} ({pct:5.1f}%)")
            else:
                print("   No failure reason data available")
        
        if has_error_message:
            # Sample error messages for top failures
            print(f"\n SAMPLE ERROR MESSAGES (Top 10):")
            print("-" * 80)
            
            error_samples = (failures_df
                           .filter(col("error_message").isNotNull() & (col("error_message") != ""))
                           .select("workspace_name", "item_name", "error_message")
                           .limit(10))
            
            error_results = error_samples.collect()
            
            if len(error_results) > 0:
                for idx, row in enumerate(error_results, 1):
                    ws_name = row['workspace_name'] if row['workspace_name'] else "Unknown"
                    item = row['item_name'] if row['item_name'] else "Unknown"
                    error = row['error_message']
                    # Truncate long messages
                    error_display = error[:100] + "..." if len(error) > 100 else error
                    print(f"\n  {idx:2d}. Workspace: {ws_name}")
                    print(f"      Item: {item}")
                    print(f"      Error: {error_display}")
            else:
                print("   No error message data available")
        
        if not has_failure_reason and not has_error_message:
            print("\n  No failure_reason or error_message columns found in data")
        
    else:
        print("\n No failures to analyze")
    
    print(f"\n" + "=" * 80)

else:
    print(" complete_df not available")

In [ ]:
# 9. Time-Based Analysis (Date & Duration)
print("=" * 80)
print(" TIME-BASED ACTIVITY ANALYSIS")
print("=" * 80)

if complete_df:
    from pyspark.sql.functions import col, count, desc, avg, sum as spark_sum, when, max as spark_max, min as spark_min
    
    # Activities by date
    print(f"\n ACTIVITY DISTRIBUTION BY DATE:")
    print("-" * 80)
    
    date_activity = (complete_df
                    .filter(col("date").isNotNull())
                    .groupBy("date")
                    .agg(
                        count("*").alias("total_activities"),
                        spark_sum(when(col("status") == "Failed", 1).otherwise(0)).alias("failures"),
                        spark_sum(when(col("status") == "Succeeded", 1).otherwise(0)).alias("successes")
                    )
                    .orderBy(desc("date"))
                    .limit(15))
    
    date_results = date_activity.collect()
    
    if len(date_results) > 0:
        for row in date_results:
            date_val = row['date']
            total = row['total_activities']
            failures = row['failures']
            successes = row['successes']
            success_rate = (successes / total * 100) if total > 0 else 0
            print(f"  {date_val}  {total:8,} total  {successes:8,} success  {failures:6,} failed  {success_rate:5.1f}% success")
    else:
        print("   No date information available")
    
    # Duration analysis
    print(f"\n⏱  DURATION ANALYSIS:")
    print("-" * 80)
    
    duration_df = complete_df.filter(col("duration_seconds").isNotNull() & (col("duration_seconds").cast("double") > 0))
    duration_count = duration_df.count()
    total_records = complete_df.count()
    
    if duration_count > 0:
        print(f"  Activities with duration data: {duration_count:,} ({duration_count/total_records*100:.1f}%)")
        
        # Overall duration statistics
        duration_stats = duration_df.agg(
            avg(col("duration_seconds").cast("double")).alias("avg_duration"),
            spark_max(col("duration_seconds").cast("double")).alias("max_duration"),
            spark_min(col("duration_seconds").cast("double")).alias("min_duration")
        ).collect()[0]
        
        print(f"\n  Overall Duration Statistics:")
        print(f"    Average: {duration_stats['avg_duration']:.2f}s ({duration_stats['avg_duration']/60:.2f} minutes)")
        print(f"    Maximum: {duration_stats['max_duration']:.2f}s ({duration_stats['max_duration']/60:.2f} minutes)")
        print(f"    Minimum: {duration_stats['min_duration']:.2f}s")
        
        # Duration by status
        print(f"\n  Duration by Status:")
        duration_by_status = (duration_df
                             .groupBy("status")
                             .agg(
                                 count("*").alias("count"),
                                 avg(col("duration_seconds").cast("double")).alias("avg_duration")
                             )
                             .orderBy(desc("count")))
        
        status_duration_results = duration_by_status.collect()
        for row in status_duration_results:
            status = row['status'] if row['status'] else "Unknown"
            count_val = row['count']
            avg_dur = row['avg_duration']
            print(f"    {status:15s}: {count_val:8,} activities, avg {avg_dur:.2f}s")
        
        # Longest running activities
        print(f"\n   TOP 10 LONGEST RUNNING ACTIVITIES:")
        longest_activities = (duration_df
                             .select("workspace_name", "item_name", "activity_type", "status", 
                                   col("duration_seconds").cast("double").alias("duration"))
                             .orderBy(desc("duration"))
                             .limit(10))
        
        longest_results = longest_activities.collect()
        for idx, row in enumerate(longest_results, 1):
            ws_name = row['workspace_name'] if row['workspace_name'] else "Unknown"
            item = row['item_name'] if row['item_name'] else "Unknown"
            activity = row['activity_type'] if row['activity_type'] else "Unknown"
            status = row['status']
            duration = row['duration']
            duration_min = duration / 60
            print(f"    {idx:2d}. {ws_name:30s}  {item:25s}  {duration:.1f}s ({duration_min:.1f}m) [{status}]")
    else:
        print("   No duration data available")
    
    print(f"\n" + "=" * 80)

else:
    print(" complete_df not available")

In [ ]:
# Import the schema definition
import sys
import os
from pathlib import Path
import usf_fabric_monitoring

try:
    from usf_fabric_monitoring.core.schema import FabricSemanticModel, ALL_DDLS
except ModuleNotFoundError:
    print("Module not found in installed package. Attempting to patch package path...")
    
    # 1. Find local src directory
    current_dir = Path(os.getcwd())
    if current_dir.name == "notebooks":
        src_path = current_dir.parent / "src"
    else:
        src_path = current_dir / "src"
        
    # 2. Add to sys.path if missing
    if src_path.exists() and str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
        print(f"Added {src_path} to sys.path")
        
    # 3. CRITICAL: Patch the already loaded package's __path__
    # This tells Python to look for submodules in the local folder too
    local_package_path = src_path / "usf_fabric_monitoring"
    if local_package_path.exists():
        # Convert to string for compatibility
        local_path_str = str(local_package_path)
        if local_path_str not in usf_fabric_monitoring.__path__:
            usf_fabric_monitoring.__path__.insert(0, local_path_str)
            print(f"Patched usf_fabric_monitoring.__path__ with: {local_path_str}")
            
            # Also need to patch 'core' if it's already loaded
            if hasattr(usf_fabric_monitoring, 'core') and hasattr(usf_fabric_monitoring.core, '__path__'):
                local_core_path = local_package_path / "core"
                if str(local_core_path) not in usf_fabric_monitoring.core.__path__:
                    usf_fabric_monitoring.core.__path__.insert(0, str(local_core_path))
                    print(f"Patched usf_fabric_monitoring.core.__path__")

    # Retry import
    from usf_fabric_monitoring.core.schema import FabricSemanticModel, ALL_DDLS

# Initialize the model
model = FabricSemanticModel()

# Print the Semantic Model Description
print(model.describe())

# Example: Print DDL for Activities Master
print("\nDDL for Activities Master:")
print(ALL_DDLS["activities_master"])